In [65]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [66]:
new_allegations = pd.read_csv("/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/2010-2014/allegations.csv", low_memory=False)
old_allegations = pd.read_csv("/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/2000-2009/00-09_allegations.csv", low_memory=False)
officer_profile = pd.read_csv("/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/2000-2009/00-09_officer_profile.csv", low_memory=False)
historic = pd.read_csv("/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/2000-2009/00-09_beat_counts.csv", low_memory=False)

In [67]:
new_allegations.columns, old_allegations.columns, officer_profile.columns, historic.columns

(Index(['CRID', 'OfficerID', 'OfficeFirst', 'OfficerLast', 'AllegationCode',
        'Category', 'Allegation', 'RecommendedFinding', 'RecommendedOutcome',
        'FinalFinding', 'FinalOutcome', 'Finding', 'Outcome', 'Beat',
        'Location', 'Add1', 'Add2', 'City', 'IncidentDate', 'StartDate',
        'EndDate', 'InvestigatorName', 'InvestigatorRank', 'Latitude',
        'Longitude'],
       dtype='object'),
 Index(['CRID', 'OfficerID', 'OfficeFirst', 'OfficerLast', 'AllegationCode',
        'Category', 'Allegation', 'RecommendedFinding', 'RecommendedOutcome',
        'FinalFinding', 'FinalOutcome', 'Finding', 'Outcome', 'Beat',
        'Location', 'Add1', 'Add2', 'City', 'IncidentDate', 'StartDate',
        'EndDate', 'InvestigatorName', 'InvestigatorRank', 'Latitude',
        'Longitude'],
       dtype='object'),
 Index(['OfficerID', 'OfficerFirst', 'OfficerLast', 'Gender', 'Race',
        'ApptDate', 'Unit', 'Rank', 'Star', 'Age'],
       dtype='object'),
 Index(['Unnamed: 0', 'B

In [68]:
# map the Beat from old_allegations to officers in officer_profile using OfficerID
officer_profile["Beat"] = old_allegations[["Beat", "OfficerID"]].groupby("OfficerID").first().reset_index()["Beat"]

In [69]:
# map historic Counts to officers in officer_profile using Beat
officer_profile["historic_counts"] = historic[["Count", "Beat"]].groupby("Beat").first().reset_index()["Count"]
officer_profile["historic_counts"] = officer_profile["historic_counts"].fillna(0)
officer_profile.head(10)

,OfficerID,OfficerFirst,OfficerLast,Gender,Race,ApptDate,Unit,Rank,Star,Age,Beat,historic_counts
0,4,Carmel,Abbate,M,White,1/6/69,NaN,Po As Detective,NaN,80,321.0,71.0
1,13,Dale,Abbott,M,White,11/29/82,NaN,Police Officer,NaN,69,1511.0,75.0
2,17,Moulay,Abdullah,M,Black,12/26/89,NaN,Police Officer,NaN,64,2525.0,260.0
3,19,Restituto,Abejero,M,Asian/Pacific,12/13/99,NaN,Police Officer,NaN,73,1122.0,2.0
4,34,Floyd,Abron,M,Black,6/29/98,NaN,Police Officer,NaN,49,1212.0,3.0
5,38,Abdalla,Abuzanat,M,Asian/Pacific,6/29/98,NaN,Po As Detective,NaN,51,1121.0,128.0
6,44,Marco,Acevedo,M,Hispanic,9/5/95,NaN,Po As Detective,NaN,53,2411.0,143.0
7,58,Manuel,Acevedo,M,Hispanic,7/19/82,NaN,Police Officer,NaN,66,1651.0,115.0
8,59,Edward,Acevedo,M,Hispanic,6/5/95,NaN,Police Officer,NaN,59,621.0,164.0
9,61,Martin,Acevedo,M,Hispanic,5/10/99,NaN,Police Officer,NaN,49,2024.0,787.0


In [70]:
# get list of unique officers
officers = list(set(new_allegations['OfficerID'].unique()).union(set(old_allegations['OfficerID'].unique())))

In [71]:
# if officer not in new_allegations, drop row
print(new_allegations.shape)
new_allegations = new_allegations[new_allegations['OfficerID'].isin(officers)]
print(new_allegations.shape)

(29496, 25)
(29496, 25)


In [72]:
# if officer not in old_allegations, drop row
print(old_allegations.shape)
old_allegations = old_allegations[old_allegations['OfficerID'].isin(officers)]
print(old_allegations.shape)

(102224, 25)
(102224, 25)


In [73]:
prev_allegations = old_allegations.groupby('OfficerID').size().reset_index(name='prev_allegations')
officer_profile["prev_allegations"] = officer_profile["OfficerID"].map(prev_allegations.set_index('OfficerID')['prev_allegations'])
officer_profile.head(10)

,OfficerID,OfficerFirst,OfficerLast,Gender,Race,ApptDate,Unit,Rank,Star,Age,Beat,historic_counts,prev_allegations
0,4,Carmel,Abbate,M,White,1/6/69,NaN,Po As Detective,NaN,80,321.0,71.0,2
1,13,Dale,Abbott,M,White,11/29/82,NaN,Police Officer,NaN,69,1511.0,75.0,2
2,17,Moulay,Abdullah,M,Black,12/26/89,NaN,Police Officer,NaN,64,2525.0,260.0,25
3,19,Restituto,Abejero,M,Asian/Pacific,12/13/99,NaN,Police Officer,NaN,73,1122.0,2.0,4
4,34,Floyd,Abron,M,Black,6/29/98,NaN,Police Officer,NaN,49,1212.0,3.0,11
5,38,Abdalla,Abuzanat,M,Asian/Pacific,6/29/98,NaN,Po As Detective,NaN,51,1121.0,128.0,13
6,44,Marco,Acevedo,M,Hispanic,9/5/95,NaN,Po As Detective,NaN,53,2411.0,143.0,7
7,58,Manuel,Acevedo,M,Hispanic,7/19/82,NaN,Police Officer,NaN,66,1651.0,115.0,6
8,59,Edward,Acevedo,M,Hispanic,6/5/95,NaN,Police Officer,NaN,59,621.0,164.0,10
9,61,Martin,Acevedo,M,Hispanic,5/10/99,NaN,Police Officer,NaN,49,2024.0,787.0,7


In [74]:
new_allegations = new_allegations.groupby('OfficerID').size().reset_index(name='new_allegations')
officer_profile["new_allegations"] = officer_profile["OfficerID"].map(new_allegations.set_index('OfficerID')['new_allegations'])
officer_profile.head(10)

,OfficerID,OfficerFirst,OfficerLast,Gender,Race,ApptDate,Unit,Rank,Star,Age,Beat,historic_counts,prev_allegations,new_allegations
0,4,Carmel,Abbate,M,White,1/6/69,NaN,Po As Detective,NaN,80,321.0,71.0,2,NaN
1,13,Dale,Abbott,M,White,11/29/82,NaN,Police Officer,NaN,69,1511.0,75.0,2,1.0
2,17,Moulay,Abdullah,M,Black,12/26/89,NaN,Police Officer,NaN,64,2525.0,260.0,25,NaN
3,19,Restituto,Abejero,M,Asian/Pacific,12/13/99,NaN,Police Officer,NaN,73,1122.0,2.0,4,1.0
4,34,Floyd,Abron,M,Black,6/29/98,NaN,Police Officer,NaN,49,1212.0,3.0,11,1.0
5,38,Abdalla,Abuzanat,M,Asian/Pacific,6/29/98,NaN,Po As Detective,NaN,51,1121.0,128.0,13,NaN
6,44,Marco,Acevedo,M,Hispanic,9/5/95,NaN,Po As Detective,NaN,53,2411.0,143.0,7,1.0
7,58,Manuel,Acevedo,M,Hispanic,7/19/82,NaN,Police Officer,NaN,66,1651.0,115.0,6,NaN
8,59,Edward,Acevedo,M,Hispanic,6/5/95,NaN,Police Officer,NaN,59,621.0,164.0,10,NaN
9,61,Martin,Acevedo,M,Hispanic,5/10/99,NaN,Police Officer,NaN,49,2024.0,787.0,7,1.0


In [75]:
# if new_allgations is NaN, set it to 0
officer_profile['new_allegations'] = officer_profile['new_allegations'].fillna(0)
officer_profile.head(10)

,OfficerID,OfficerFirst,OfficerLast,Gender,Race,ApptDate,Unit,Rank,Star,Age,Beat,historic_counts,prev_allegations,new_allegations
0,4,Carmel,Abbate,M,White,1/6/69,NaN,Po As Detective,NaN,80,321.0,71.0,2,0.0
1,13,Dale,Abbott,M,White,11/29/82,NaN,Police Officer,NaN,69,1511.0,75.0,2,1.0
2,17,Moulay,Abdullah,M,Black,12/26/89,NaN,Police Officer,NaN,64,2525.0,260.0,25,0.0
3,19,Restituto,Abejero,M,Asian/Pacific,12/13/99,NaN,Police Officer,NaN,73,1122.0,2.0,4,1.0
4,34,Floyd,Abron,M,Black,6/29/98,NaN,Police Officer,NaN,49,1212.0,3.0,11,1.0
5,38,Abdalla,Abuzanat,M,Asian/Pacific,6/29/98,NaN,Po As Detective,NaN,51,1121.0,128.0,13,0.0
6,44,Marco,Acevedo,M,Hispanic,9/5/95,NaN,Po As Detective,NaN,53,2411.0,143.0,7,1.0
7,58,Manuel,Acevedo,M,Hispanic,7/19/82,NaN,Police Officer,NaN,66,1651.0,115.0,6,0.0
8,59,Edward,Acevedo,M,Hispanic,6/5/95,NaN,Police Officer,NaN,59,621.0,164.0,10,0.0
9,61,Martin,Acevedo,M,Hispanic,5/10/99,NaN,Police Officer,NaN,49,2024.0,787.0,7,1.0


In [76]:
# drop columns Unit, Star
officer_profile = officer_profile.drop(columns=['Unit', 'Star'])
officer_profile.head(10)

,OfficerID,OfficerFirst,OfficerLast,Gender,Race,ApptDate,Rank,Age,Beat,historic_counts,prev_allegations,new_allegations
0,4,Carmel,Abbate,M,White,1/6/69,Po As Detective,80,321.0,71.0,2,0.0
1,13,Dale,Abbott,M,White,11/29/82,Police Officer,69,1511.0,75.0,2,1.0
2,17,Moulay,Abdullah,M,Black,12/26/89,Police Officer,64,2525.0,260.0,25,0.0
3,19,Restituto,Abejero,M,Asian/Pacific,12/13/99,Police Officer,73,1122.0,2.0,4,1.0
4,34,Floyd,Abron,M,Black,6/29/98,Police Officer,49,1212.0,3.0,11,1.0
5,38,Abdalla,Abuzanat,M,Asian/Pacific,6/29/98,Po As Detective,51,1121.0,128.0,13,0.0
6,44,Marco,Acevedo,M,Hispanic,9/5/95,Po As Detective,53,2411.0,143.0,7,1.0
7,58,Manuel,Acevedo,M,Hispanic,7/19/82,Police Officer,66,1651.0,115.0,6,0.0
8,59,Edward,Acevedo,M,Hispanic,6/5/95,Police Officer,59,621.0,164.0,10,0.0
9,61,Martin,Acevedo,M,Hispanic,5/10/99,Police Officer,49,2024.0,787.0,7,1.0


In [ ]:
# export to csv
officer_profile.to_csv("/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Processed/2010-2014_train/officer_profile.csv", index=False)